In [11]:
# pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyspark

import time

In [3]:
import pandas as pd
from functools import reduce

# Pandas

In [4]:
# Leitura dos ficheiros csv
file1 = pd.read_csv("C:\\Users\\carla\\OneDrive\\Ambiente de Trabalho\\Universidade\Mestrado\\1º Ano\\2º Semestre\\Big Data\\Projeto\\Data Sets\\death-rate-from-suicides-gho.csv")
file2 = pd.read_csv("C:\\Users\\carla\\OneDrive\\Ambiente de Trabalho\\Universidade\Mestrado\\1º Ano\\2º Semestre\\Big Data\\Projeto\\Data Sets\\death-rates-from-drug-use-disorders.csv")
file3 = pd.read_csv("C:\\Users\\carla\\OneDrive\\Ambiente de Trabalho\\Universidade\Mestrado\\1º Ano\\2º Semestre\\Big Data\\Projeto\\Data Sets\\total-alcohol-consumption-per-capita-litres-of-pure-alcohol.csv")
file4 = pd.read_csv("C:\\Users\\carla\\OneDrive\\Ambiente de Trabalho\\Universidade\Mestrado\\1º Ano\\2º Semestre\\Big Data\\Projeto\\Data Sets\\burden-disease-from-each-mental-illness.csv")

In [28]:
start = time.time()

Data_Frames = [file1, file2, file3, file4]
combdados = reduce(lambda left, right:pd.merge(left, right, on=['Entity','Year','Code']), Data_Frames)
paises_comuns = combdados['Entity'].unique()
anos_comuns = combdados['Year'].unique()
df_final = combdados[combdados['Entity'].isin(paises_comuns) & combdados['Year'].isin(anos_comuns)]

end = time.time()

tempo_pandas = (end - start)
print(tempo_pandas)
df_final

0.020047903060913086


,Entity,Code,Year,Age-standardized suicide rate - Sex: both sexes,Deaths - Drug use disorders - Sex: Both - Age: Age-standardized (Rate),"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)","DALYs from depressive disorders per 100,000 people in, both sexes aged age-standardized","DALYs from schizophrenia per 100,000 people in, both sexes aged age-standardized","DALYs from bipolar disorder per 100,000 people in, both sexes aged age-standardized","DALYs from eating disorders per 100,000 people in, both sexes aged age-standardized","DALYs from anxiety disorders per 100,000 people in, both sexes aged age-standardized"
0,Afghanistan,AFG,2010,6.6700,1.55,0.21,888.55730,133.93993,147.637950,21.996126,442.29670
1,Afghanistan,AFG,2015,5.9900,1.55,0.21,889.83100,135.82290,148.000870,24.176653,453.54910
2,Afghanistan,AFG,2018,5.9100,1.60,0.21,888.19760,135.77147,148.186950,24.742624,455.19724
3,Albania,ALB,2000,5.2300,0.72,6.57,383.35883,181.26312,117.579150,22.061806,355.20570
4,Albania,ALB,2005,7.6800,0.78,7.65,388.85956,182.34738,117.714270,24.131720,356.89362
...,...,...,...,...,...,...,...,...,...,...,...
899,Zimbabwe,ZWE,2000,19.9764,1.07,2.46,522.54120,129.83534,114.570190,23.274557,302.44020
900,Zimbabwe,ZWE,2005,21.9666,1.06,2.77,534.28345,128.15717,114.573400,21.463827,302.60360
901,Zimbabwe,ZWE,2010,35.0142,1.16,3.93,530.62470,126.10969,114.908585,19.506409,301.86316
902,Zimbabwe,ZWE,2015,30.7352,1.26,4.92,546.46204,127.10872,115.320730,20.423056,302.20868


In [49]:
z = df_final.Entity.value_counts() < 5

if z.any():
    df_final = df_final[~df_final.Entity.isin(z[z].index)]
else:
    pass


df_final

,Entity,Code,Year,Age-standardized suicide rate - Sex: both sexes,Deaths - Drug use disorders - Sex: Both - Age: Age-standardized (Rate),"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)","DALYs from depressive disorders per 100,000 people in, both sexes aged age-standardized","DALYs from schizophrenia per 100,000 people in, both sexes aged age-standardized","DALYs from bipolar disorder per 100,000 people in, both sexes aged age-standardized","DALYs from eating disorders per 100,000 people in, both sexes aged age-standardized","DALYs from anxiety disorders per 100,000 people in, both sexes aged age-standardized"
3,Albania,ALB,2000,5.2300,0.72,6.57,383.35883,181.26312,117.579150,22.061806,355.20570
4,Albania,ALB,2005,7.6800,0.78,7.65,388.85956,182.34738,117.714270,24.131720,356.89362
5,Albania,ALB,2010,7.6300,0.68,7.69,383.14902,184.12775,117.873230,26.319311,360.68094
6,Albania,ALB,2015,4.2300,0.82,6.74,382.93760,185.51762,117.718880,28.092310,366.80685
7,Albania,ALB,2018,3.9400,0.87,7.17,384.17953,185.36105,117.761635,28.972872,366.46720
...,...,...,...,...,...,...,...,...,...,...,...
899,Zimbabwe,ZWE,2000,19.9764,1.07,2.46,522.54120,129.83534,114.570190,23.274557,302.44020
900,Zimbabwe,ZWE,2005,21.9666,1.06,2.77,534.28345,128.15717,114.573400,21.463827,302.60360
901,Zimbabwe,ZWE,2010,35.0142,1.16,3.93,530.62470,126.10969,114.908585,19.506409,301.86316
902,Zimbabwe,ZWE,2015,30.7352,1.26,4.92,546.46204,127.10872,115.320730,20.423056,302.20868


In [52]:
df_final.Entity.value_counts() == 5

Entity
Albania        True
Rwanda         True
Netherlands    True
New Zealand    True
Nicaragua      True
               ... 
Germany        True
Ghana          True
Greece         True
Grenada        True
Zimbabwe       True
Name: count, Length: 178, dtype: bool

# PySpark

In [8]:
# É necessário criar uma sessão antes de usar o PySpark

import pyspark
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder.appName('Projeto').getOrCreate()

In [10]:
# Ver qual a versão do spark e a sessão em que estamos
spark

In [11]:
# Import dos dataframes
# O comando 'option(header, true)' é importante para definir o título das colunas, que por omissão seria c_0, c_1, etc...
# inferSchema=True é importante para transformar os dados que são assumidos como str em int

df_file1 = spark.read.option('header','true').csv("C:\\Users\\carla\\OneDrive\\Ambiente de Trabalho\\Universidade\Mestrado\\1º Ano\\2º Semestre\\Big Data\\Projeto\\Data Sets\\death-rate-from-suicides-gho.csv",inferSchema = True)
df_file2 = spark.read.option('header','true').csv("C:\\Users\\carla\\OneDrive\\Ambiente de Trabalho\\Universidade\Mestrado\\1º Ano\\2º Semestre\\Big Data\\Projeto\\Data Sets\\death-rates-from-drug-use-disorders.csv",inferSchema = True)
df_file3 = spark.read.option('header','true').csv("C:\\Users\\carla\\OneDrive\\Ambiente de Trabalho\\Universidade\Mestrado\\1º Ano\\2º Semestre\\Big Data\\Projeto\\Data Sets\\total-alcohol-consumption-per-capita-litres-of-pure-alcohol.csv",inferSchema = True)
df_file4 = spark.read.option('header','true').csv("C:\\Users\\carla\\OneDrive\\Ambiente de Trabalho\\Universidade\Mestrado\\1º Ano\\2º Semestre\\Big Data\\Projeto\\Data Sets\\burden-disease-from-each-mental-illness.csv",inferSchema = True)

In [12]:
# Verificar o tipo de data presente no ficheiro csv
type(df_file1)

pyspark.sql.dataframe.DataFrame

In [13]:
df_file1.printSchema()

# se não especificarmos, o tipo de dado vai ser assumido como string

root
 |-- Entity: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Age-standardized suicide rate - Sex: both sexes: double (nullable = true)



In [14]:
# Mostra o tipo de dados presentes nos df
df_file1

# Mostra as primeiras 20 linhas (por predefinição) presentes nos df
df_file1.show(5)

+-----------+----+----+-----------------------------------------------+
|     Entity|Code|Year|Age-standardized suicide rate - Sex: both sexes|
+-----------+----+----+-----------------------------------------------+
|Afghanistan| AFG|2000|                                           7.71|
|Afghanistan| AFG|2001|                                           7.89|
|Afghanistan| AFG|2002|                                           7.85|
|Afghanistan| AFG|2003|                                           7.72|
|Afghanistan| AFG|2004|                                           7.77|
+-----------+----+----+-----------------------------------------------+
only showing top 5 rows



In [15]:
start = time.time()
df_joined = df_file1.join(df_file2, ['Entity', 'Code', 'Year'],"inner").join(df_file3, ['Entity', 'Code', 'Year'],"inner").join(df_file4, ['Entity', 'Code', 'Year'],"inner")

end = time.time()

tempo_spark = (end - start)
print(tempo_spark)

0.08040761947631836


In [16]:
df_joined.show()

+-------------------+----+----+-----------------------------------------------+----------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+------------------------------------------------------------------------------------+
|             Entity|Code|Year|Age-standardized suicide rate - Sex: both sexes|Deaths - Drug use disorders - Sex: Both - Age: Age-standardized (Rate)|Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)|DALYs from depressive disorders per 100,000 people in, both sexes aged age-

In [17]:
print(f'Utilizando o PySpark demora {tempo_spark} segundos') 
print(f'Utilizando o Pandas demora {tempo_pandas} segundos') 

Utilizando o PySpark demora 0.08040761947631836 segundos
Utilizando o Pandas demora 0.031293630599975586 segundos


# Escolha da Base de Dados

Username -> carlosfcgomes0
Password -> vX471wc4u4VzMExR

IPList Acess -> 193.137.92.118/32

In [25]:
'''
Instalação do driver
'''

# pip install pymongo[srv]

'\nInstalação do driver\n'

In [27]:
'''
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
uri = "mongodb+srv://carlosfcgomes0:<vX471wc4u4VzMExR>@bigdata.ln48uuq.mongodb.net/?retryWrites=true&w=majority&appName=BigData"
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
'''

ac-ogm7ywz-shard-00-02.ln48uuq.mongodb.net:27017: [WinError 10061] Nenhuma ligação pôde ser feita porque o computador de destino
as recusou ativamente (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),ac-ogm7ywz-shard-00-00.ln48uuq.mongodb.net:27017: [WinError 10061] Nenhuma ligação pôde ser feita porque o computador de destino
as recusou ativamente (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),ac-ogm7ywz-shard-00-01.ln48uuq.mongodb.net:27017: [WinError 10061] Nenhuma ligação pôde ser feita porque o computador de destino
as recusou ativamente (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 65e9b1895d3506f662d5ea87, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-ogm7ywz-shard-00-00.ln48uuq.mongodb.net', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('ac-ogm7ywz-shard-00-00.ln48uuq.mongodb.net:27017: